# ``Assignment``: Predicting Daily Temperatures using Artificial Neural Networks

## ``Objective``:
To design and implement an artificial neural network using PyTorch to predict daily temperatures based on historical weather data. This project will involve data preprocessing, building, and training an ANN model, and evaluating its performance.

## ``Dataset``:
We will use the **Daily minimum temperatures in Melbourne dataset** from the `UCI Machine Learning Repository` or Kaggle. This dataset contains daily temperature observations for Melbourne, Australia, over a period of 10 years, 1981-1990.

## ``Importance and Analysis``

### Nature of the Project
This project focuses on time-series prediction, which is a critical task in weather data science. Weather data is naturally sequential; the temperature on one day is often influenced by temperatures on previous days. Predicting future temperatures based on past observations can help in planning and decision-making in various sectors like agriculture, energy, and public safety.

### Importance of Sequence Data
**Why Use Sequence Data?**
- In time-series prediction, sequences capture temporal dependencies in the data. By considering the temperatures of the past few days, the model can learn patterns and trends that are useful for predicting future temperatures. This sequential information is crucial for accurate predictions.

### Relevance of Sequences in Weather Data Science
Weather patterns are inherently sequential. For instance, a cold front or heatwave affects temperatures over several days. By using sequences, we allow the model to understand these patterns and make more informed predictions.

### Handling Sequences in ANNs
In this project, we will create sequences of temperature data to feed into our neural network. Here's how we'll handle sequences:

1. **Sequence Length**: We'll use the past 7 days to predict the next day's temperature. This is our sequence length.
2. **Input Shape**: Each input to the model will be a sequence of 7 days of temperature data.
3. **Output Shape**: The output will be a single value, representing the predicted temperature for the next day.

## ``Tasks``:

### Part 1: Data Loading and Preprocessing

## Step 1: Download the Dataset

Download the **Daily minimum temperatures in Melbourne dataset** from the following link:

[Kaggle: Daily Minimum Temperatures in Melbourne Dataset](https://www.kaggle.com/datasets/paulbrabban/daily-minimum-temperatures-in-melbourne)

1. Go to the provided link.
2. Click on the "Download" button to download the dataset as a CSV file.
3. Extract the CSV file from the downloaded ZIP file if it is compressed.
4. Rename the CSV file to `daily-min-temperatures.csv` if it has a different name.

#### 1. Load the Weather Dataset
Use `pandas` to load the dataset from a CSV file.

```python
import pandas as pd

# Load the dataset
df = pd.read_csv('daily-min-temperatures.csv')

# Display the first few rows of the dataframe
print(df.head()) 

#### 2. Data Cleaning

- Look at the data, you may need to remove unecessary rows (may be last useless row), additionally you may also need to prepare the dataframe properly.
- Handle missing values appropriately (e.g., imputation, removal). Check for NaN and infinities, if present use forward fill method to replace the NaN.

```python
# Fill missing values with the forward fill method
df.fillna(method='ffill', inplace=True)

#### 3. Feature Engineering 
( Although you will not be using this erived feature in the actual modelling part, this section is just to let you know the concept of derived features)
- Create new features based on the existing data (e.g., rolling averages, time-based features).

#### Why Feature Engineering?
Feature engineering helps in creating additional relevant features from the existing data, which can provide the model with more information and potentially improve its performance. For example, using rolling averages can help smooth out the data and highlight trends.

```python
# Create rolling average features (you should try to learn more about rolling average and its importance in weather)
df['Temp_rolling_mean'] = df['Temp'].rolling(window=7).mean()

# Drop rows with NaN values generated by rolling mean
df.dropna(inplace=True)

# Display the first few rows of the dataframe with the new feature
print(df.head())

#### 4. Data Normalization
Normalize the temperature values to have values between 0 and 1.

#### Why Normalize?
Normalization scales the data to a common range, which can help improve the convergence rate during training and lead to better performance.

```python
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
scaler = MinMaxScaler()

# Normalize the temperature values
df['Temp'] = scaler.fit_transform(df[['Temp']])
df['Temp_rolling_mean'] = scaler.fit_transform(df[['Temp_rolling_mean']])

# Display the first few rows of the dataframe after normalization
print(df.head())

#### 5. Sequence Generation
Create sequences of data for time-series prediction (e.g., use the past 7 days to predict the next day's temperature). Only use temperature for sequence generation, donot use rolling average.

#### Why Sequence?
Time-series data is inherently sequential, and using sequences allows the model to learn temporal dependencies. For instance, knowing the temperatures of the past 7 days helps predict the temperature for the next day.

```python
import numpy as np

def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        label = data[i + seq_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

# Define the sequence length (let say 7)
seq_length = 7

# Create sequences of data
X, y = create_sequences(df['Temp'].values, seq_length)

# Display the shape of the generated sequences
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

### Part 2: Building the Neural Network
Define the neural network architecture, implement the training loop, and train the model using the training data.

#### 1. Define the Neural Network Architecture:
* Input layer: Number of input neurons equal to the number of days used for prediction (e.g., 7).
* Hidden layers: Two hidden layers, each with 64 neurons and ReLU activation.
* Output layer: 1 neuron (for the predicted temperature).

### Part 3: Training the Neural Network
#### 1. Define the Loss Function and Optimizer:
* Use Mean Squared Error (MSE) for the loss function.
* Use Adam for the optimizer.

#### 2. Train the Network:
* Implement the training loop. Split your data into train and test set, and train on train set.
* Track the loss during training.

```python

from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X, dtype=torch.float32)
y_train = torch.tensor(y, dtype=torch.float32).view(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

### Part 4: Evaluating the Model
#### 1. Evaluate the Model on Test Data:
Calculate the Root Mean Squared Error (RMSE) of the model on the test dataset.
Plot the actual vs. predicted temperatures.

```python
# Evaluate the model
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    rmse = torch.sqrt(criterion(predictions, y_test))
    print(f"Test RMSE: {rmse.item()}")

# Plot actual vs. predicted temperatures
plt.figure(figsize=(10, 6))
plt.plot(y_test.numpy(), label='Actual')
plt.plot(predictions.numpy(), label='Predicted')
plt.legend()
plt.xlabel('Day')
plt.ylabel('Normalized Temperature')
plt.title('Actual vs. Predicted Temperatures')
plt.show()

### Part 5: Making Predictions and Analysis
#### 1. Make Predictions on New Data:
Use the trained model to make predictions on new data.
Display some example predictions with the corresponding actual temperatures.

```python
# Generate new sequences for predictions (e.g., the last 7 days in the dataset)
new_data = df['Temp'].values[-seq_length:]
new_data = torch.tensor(new_data, dtype=torch.float32).view(1, -1)

# Make predictions
model.eval()
with torch.no_grad():
    new_predictions = model(new_data)

print(f"Predicted temperature for the next day: {new_predictions.item()}")

#### 2. Analyze Model Performance:
Discuss potential improvements and further steps. (This section is not mandatory but recommended)

# ``Special section``

## Special Section 1: Analysis of Model Performance

- The RMSE on the test data provides an indication of the model's performance. 
- Compare the actual vs. predicted temperatures plot to visually assess the model's predictions.
- Potential improvements could include tuning the hyperparameters, adding more features, or using a different model architecture.
- Further steps could involve experimenting with different sequence lengths or using more advanced techniques like LSTM networks for time-series prediction.

## Special Section 2: Preparation of Train and Test Data

### Understanding Train and Test Data Preparation

In time-series prediction, the preparation of train and test data is different from typical supervised learning tasks. Here, if the input `X` is the temperature data for 7 days, the corresponding ground truth `y` is the temperature on the 8th day. This approach helps the model learn the temporal dependencies in the data.

### Why is This Important?

Time-series data has a natural order, and it is crucial to maintain this order when splitting the data into training and testing sets. Shuffling the data would break the temporal relationships, leading to poor model performance.

### Step-by-Step Process

1. **Create Sequences**: Generate sequences of data where each sequence contains data for 7 consecutive days.
2. **Split the Data**: Split the sequences into training and testing sets without shuffling to maintain the temporal order.
3. **Prepare Tensors**: Convert the sequences into PyTorch tensors for model training.

### Code Snippet for Data Preparation

```python
import numpy as np
from sklearn.model_selection import train_test_split
import torch

# Define the function to create sequences
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        label = data[i + seq_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

# Define the sequence length
seq_length = 7

# Create sequences of data
X, y = create_sequences(df['Temp'].values, seq_length)

# Split the data into training and testing sets without shuffling
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Display the shape of the train and test sets
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_test: {y_test.shape}")